#Explanation - Constraint Satisfaction Problems



In this portion of this laboratory, you are to complete the implementation of a general constraint satisfaction problem solver.

We have provided you a basic CSP implementation in the CSP code. The implementation has the Depth-first-search already completed. It even has a basic built in constraint checker. So it will produce the search trees of the kind for DFS with back tracking with basic constraint checking.

However, it doesn't do forward checking or forward checking + singleton propagation!

So your job is to complete:

    forward_checking(state):
and
     
    forward_checking_prop_singleton(state):
Here `state` is an instance of `CSPState`, an object that keeps track of the current variable assignments and domains. These functions are called by the Search algorithm at every node in the search tree. These functions should return `False` at points at which the Domain Reduction Algorithm would backtrack, and `True` otherwise (i.e. continue extending).

As a hint, here is the (unrefined) pseudocode for the two algorithms.

## Forward Checking

1.   Let X be the variable currently being assigned.
2.   Let x be the value being assigned to X.
3.   Find all the binary constraints that are associated with X.
4.   For each constraint:
      1.   Let Y be the variable connected to X by that binary constraint.
      2.   For each variable value y in Y's domain
           1. If constraint checking fails for X=x and Y=y
                 1. Remove y from Y's domain
           2. If the domain of Y is reduced down to the empty set, then the entire check fails: return False.
5.   If all constraints passed declare success, return True

If you get a state with no current variable assignment (at the Root of the search tree) then you should just return True, since forward checking could only be applied when there is some variable assignment.

## Forward Checking with Propagation through Singletons

1.   Run forward checking, fail if forward checking fails.
2.   Find variables with domains of size 1.
3.   Create a queue of singleton variables.
4.   While singleton queue is not empty
      1. Pop off the first singleton variable X (add X to list of visited singletons)
      2. Find all the binary constraints that singleton X is associated with.
      3. For each constraint therein:
           1. Let Y be the variable connected to X by that binary constraint
                1. For each value of y in Y's domain:
                    1. If constraint check fails for X = (X's singleton value) and Y = y:
                        1. Remove y from Y's domain
                    2. If the domain of Y is reduced down to the empty set, then the entire check fails, return False.
      4. Check to see if domain reduction produced any new and unvisited singletons; if so, add them to the queue.
5. return True.

## API

These are some useful functions defined in the CSP code that you should use in your code to implement the above algorithms:

`CSPState`: representation of one of the many possible search states in the CSP problem.
* `get_current_variable()` - gets the Variable instance being currently assigned. Returns `None` if we are in the root state, when there are no variable assignments yet.
* `get_constraints_by_name(variable_name)` - retrieves all the `BinaryConstraint` objects associated with `variable_name`.
* `get_variable_by_name(variable_name)` - retrieves the `Variable` object associated with `variable_name`.
* `get_all_variables()` - gets the list of all `Variable` objects in this CSP problem.

`Variable`: representation of a variable in these problems.
* `get_name()` - returns the name of this variable.
* `get_assigned_value()` - returns the **assigned** value of this variable. **Returns `None` if `is_assigned()` returns `False`, that is if the variable hasn't been assigned yet.**
* `is_assigned()` - returns `True` if we've made an assignment for this variable.
* `get_domain()` - returns a copy of the list of the current domain of this variable. Use this to iterate over values of Y.

**You might want to consider using this method to get the singular value of a variable with domain size reduced to 1.**
* `reduce_domain(value)` - remove `value` from this variable's domain.
* `domain_size()` - returns the size of this variable's domain

`BinaryConstraint`: a binary constraint on variable i, j: i -> j.
* `get_variable_i_name()` - name of the i variable
* `get_variable_j_name()` - name of the j variable
* `check(state, value_i=value, value_j=value)` - checks the binary constraint for a given CSP state, with variable i set by value i, and variable j set by value j. Returns `False` if the constraint fails. Raises an exception if `value_i` or `value_j` are not set or cannot be inferred from state.

NOTE: in our implementation of CSPs, constraints are symmetrical; a constraint object exists for each "direction" of a constraint, so you can check for the presence of a constraint by substituting for i and/or j in the most convenient fashion for you.

Here is how you might use the API to get the value of a variable currently being assigned.

    var = state.get_current_variable()
    value = None
    if var is not None: # we are not in the root state
       value = var.get_assigned_value()
       # Here value is the value of the variable current being assigned.
Here is how you might use the API to get the singular value from a singleton variable:
         
    if singleton_var.domain_size() == 1
    value = singleton_var.get_domain()[0]
    
## Testing

For unit testing, we have provided the  Moose CSP code, an implementation of the seating problem involving a Moose, Palin, McCain, Obama, Biden, and You -- in terms of the framework as defined in the CSP code.

Running:

    solve_csp_problem(moose_csp_problem, checker, verbose=True)
will return the search tree for DFS with constraint checking. When you have finished your implementation, running `solve_csp_problem(moose_csp_problem, checker=forward_checking, verbose=True)` or `(moose_csp_problem, checker=forward_checking_prop_singleton, verbose=True)` should return the correct search trees under forward checking and forward checking with singleton propagation.

Similarly

Running:
      
    solve_csp_problem(map_coloring_csp_problem, checker, verbose=True)
Should return the expected search trees for the B,Y,R, state coloring problem.








#Explanation - Learning



Now for something completely different: learning!

## Classifying Congress

During Obama's visit to MIT, you got a chance to impress him with your analytical thinking. Now, he has hired you to do some political modeling for him. He seems to surround himself with smart people that way.

He takes a moment out of his busy day to explain what you need to do. "I need a better way to tell which of my plans are going to be supported by Congress," he explains. "Do you think we can get a model of Democrats and Republicans in Congress, and which votes separate them the most?"

"Yes, we can!" You answer.

## The Data

You acquire the data on how everyone in the previous Senate and House of Representatives voted on every issue. (These data are available in machine-readable form via voteview.com. We've included it in the laboratory directory, in the files beginning with `H110` and `S110`.)

The Data Reader code contains functions for reading data in this format.

`read_congress_data("FILENAME.ord")` reads a specially-formatted file that gives information about each Congressperson and the votes they cast. It returns a list of dictionaries, one for each member of Congress, including the following items:
* 'name': The name of the Congressperson.
* 'state': The state they represent.
* 'party': The party that they were elected under.
* 'votes': The votes that they cast, as a list of numbers. 1 represents a "yea" vote, -1 represents "nay", and 0 represents either that they abstained, were absent, or were not a member of Congress at the time.

To make sense of the votes, you will also need information about what they were voting on. This is provided by `read_vote_data("FILENAME.csv")`, which returns a list of votes in the same order that they appear in the Congresspeople's entries. Each vote is represented a dictionary of information, which you can convert into a readable string by running `vote_info(vote)`.

The Data Reader code reads in the provided data, storing them in the variables `senate_people`, `senate_votes`, `house_people`, and `house_votes`.

## Nearest Neighbors

You decide to start by making a nearest-neighbors classifier that can tell Democrats apart from Republicans in the Senate.

We've provided a `nearest_neighbors` function that classifies data based on training data and a distance function. In particular, this is a third-order function:
* First, call `nearest_neighbors(distance, k)`, with distance being the distance function you wish to use and `k` being the number of neighbors to check. This returns a *classifier factory*.
* A classifier factory is a function that makes classifiers. You call it with some training data as an argument, and it returns a classifier.
* Finally, you call the classifier with a data point (here, a Congressperson) and it returns the classification as a string.

Much of this is handled by the `evaluate(factory, group1, group2)` function, which you can use to test the effectiveness of a classification strategy. You give it a classifier factory (as defined above) and two sets of data. It will train a classifier on one data set and test the results against the other, and then it will switch them and test again.

Given a list of data such as `senate_people`, you can divide it arbitrarily into two groups using the `crosscheck_groups(data)` function.

One way to measure the "distance" between Congresspeople is with the *Hamming distance*: the number of entries that differ. This function is provided as `hamming_distance`.

An example of putting this all together is provided in the laboratory code:

    senate_group1, senate_group2 = crosscheck_groups(senate_people)
    evaluate(nearest_neighbors(edit_distance, 1), senate_group1, senate_group2, verbose=1)
Examine the results of this evaluation. In addition to the problems caused by independents, it's classifying Senator Johnson from South Dakota as a Republican instead of a Democrat, mainly because he missed a lot of votes while he was being treated for cancer. This is a problem with the distance function -- when one Senator votes yes and another is absent, that is less of a "disagreement" than when one votes yes and the other votes no.

You should address this. Euclidean distance is a reasonable measure for the distance between lists of discrete numeric features, and is the alternative to Hamming distance that you decide to try. Recall that the formula for Euclidean distance is:

*[(x1 - y1)^2 + (x2 - y2)^2 + ... + (xn - yn)^2] ^ (1/2)*
* Make a distance function called euclidean_distance that treats the votes as high-dimensional vectors, and returns the Euclidean distance between them.

When you evaluate using `euclidean_distance`, you should get better results, except that some people are being classified as Independents. Given that there are only 2 Independents in the Senate, you want to avoid classifying someone as an Independent just because they vote similarly to one of them.
* Make a simple change to the parameters of `nearest_neighbors` that accomplishes this, and call the classifier factory it outputs `my_classifier`.

## ID Trees

So far you've classified Democrats and Republicans, but you haven't created a model of which votes distinguish them. You want to make a classifier that explains the distinctions it makes, so you decide to use an ID-tree classifier.

`idtree_maker(votes, disorder_metric)` is a third-order function similar to `nearest_neighbors`. You initialize it by giving it a list of vote information (such as `senate_votes` or `house_votes`) and a function for calculating the disorder of two classes. It returns a classifier factory that will produce instances of the `CongressIDTree` class, defined in the Classify code, to distinguish legislators based on their votes.

The possible decision boundaries used by `CongressIDTree` are, for each vote:
* Did this legislator vote YES on this vote, or not?
* Did this legislator vote NO on this vote, or not?

(These are different because it is possible for a legislator to abstain or be absent.)

You can also use `CongressIDTree` directly to make an ID tree over the entire data set.

If you `print` a `CongressIDTree`, then you get a text representation of the tree. Each level of the ID tree shows the minimum disorder it found, the criterion that gives this minimum disorder, and (marked with a +) the decision it makes for legislators who match the criterion, and (marked with a -) the decision for legislators who don't. The decisions are either a party name or another ID tree. An example is shown in the section below.

## An ID tree for the entire Senate

You start by making an ID tree for the entire Senate. This doesn't leave you anything to test it on, but it will show you the votes that distinguish Republicans from Democrats the most quickly overall. You run this (which you can uncomment in your lab file):

`print(CongressIDTree(senate_people, senate_votes, homogeneous_disorder))`

The ID tree you get here is:

    Disorder: -49
    Yes on S.Con.Res. 21: Kyl Amdt. No. 583; To reform the death tax by setting the
    exemption at 5 million dollars per estate, indexed for inflation, and the top death
    tax rate at no more than 35% beginning in 2010; to avoid subjecting an
    estimated 119,200 families, family businesses, and family farms to the death
    tax each and every year; to promote continued economic growth and job creation;
    and to make the enhanced teacher deduction permanent.:
    + Republican
    - Disorder: -44
      Yes on H.R. 1585: Feingold Amdt. No. 2924; To safely redeploy United States
      troops from Iraq.:
      + Democrat
      - Disorder: -3
        No on H.R. 1495: Coburn Amdt. No. 1089; To prioritize Federal spending to
        ensure the needs of Louisiana residents who lost their homes as a result of
        Hurricane Katrina and Rita are met before spending money to design or
        construct a nonessential visitors center.:
        + Democrat
        - Disorder: -2
          Yes on S.Res. 19: S. Res. 19; A resolution honoring President Gerald
          Rudolph Ford.:
          + Disorder: -4
            Yes on H.R. 6: Motion to Waive C.B.A. re: Inhofe Amdt. No. 1666; To
            ensure agricultural equity with respect to the renewable fuels standard.:
            + Democrat
            - Independent
          - Republican
Some things that you can observe from these results are:
* Senators like to write bills with very long-winded titles that make political points.
* The key issue that most clearly divided Democrats and Republicans was the issue that Democrats call the "estate tax" and Republicans call the "death tax", with 49 Republicans voting to reform it.
* The next key issue involved 44 Democrats voting to redeploy troops from Iraq.
* The issues below that serve only to peel off homogenous groups of 2 to 4 people.

## Implementing a better disorder metric

You should be able to reduce the depth and complexity of the tree, by changing the disorder metric from the one that looks for the largest homogeneous group to the information-theoretical metric.
* Write the `information_disorder(group1, group2)` function to replace `homogeneous_disorder`. This function takes in the lists of classifications that fall on each side of the decision boundary, and returns the information-theoretical disorder.

Example:

    information_disorder(["Democrat", "Democrat", "Democrat"], ["Republican", "Republican"])
      => 0.0
    information_disorder(["Democrat", "Republican"], ["Republican", "Democrat"])
      => 1.0
Once this is written, you can try making a new `CongressIDTree` with it. (if you're having trouble, keep in mind you should return a float or similar)

## Evaluating over the House of Representatives

Now, you decide to evaluate how well ID trees do in the wild, weird world of the House of Representatives.

You can try running an ID tree on the entire House and all of its votes. It's disappointing. The 110th House began with a vote on the rules of order, where everyone present voted along straight party lines.

It's not a very informative result to observe that Democrats think Democrats should make the rules and Republicans think Republicans should make the rules.

Anyway, since your task was to make a tool for classifying the newly-elected Congress, you'd like it to work after a relatively small number of votes. We've provided a function, `limited_house_classifier`, which evaluates an ID tree classifier that uses only the most recent N votes in the House of Representatives. You just need to find a good value of *N*.
* Using `limited_house_classifier`, find a good number *N_1* of votes to take into account, so that the resulting ID trees classify at least 430 Congresspeople correctly. How many training examples (previous votes) does it take to predict at least 90 senators correctly? What about 95? **To pass the tests**, you will need to find close to the minimum such values for *N_1*, *N_2*, and *N_3*. Keep guessing to find close to the minimum that will pass the offline tests. Do the values surprise you? Is the house more unpredictable than the senate, or is it just bigger?
* Which is better at predicting the senate, 200 training samples, or 2000? Why?

The total number of Congresspeople in the evaluation may change, as people who didn't vote in the last *N* votes (perhaps because they're not in office anymore) aren't included.


#CSP

In [ ]:
"""
A General Constraint Satisfaction Problem Solver
"""
class Variable:
    """
    Representation of a discrete variable with a finite domain.
    As used in our VD table.
    A variable can be in the assigned state, in which v.is_assigned()
    will return true.
    """
    def __init__(self, name, domain, value=None):
        self._name = name
        self._domain = domain[:]
        self._value = value

    def copy(self):
        return Variable(self._name, self._domain, self._value)

    def get_name(self):
        return self._name
    
    def reduce_domain(self, value):
        self._domain.remove(value)

    def domain_size(self):
        return len(self._domain)

    def get_domain(self):
        return self._domain[:]
    
    def is_assigned(self):
        return self._value is not None

    def get_assigned_value(self):
        return self._value
    
    def set_value(self, value):
        self._value = value
        
    def __str__(self):
        buf = "%s(%s)" %(self._name, self._domain)
        if self._value is not None:
            buf += ": %s" %(self._value)
        return buf
        
class BinaryConstraint:
    """
    Representation of a binary-constraint on two variables variable i and
    variable j.
    """
    def __init__(self, var_i_name, var_j_name, check_func, description=None):
        """
        * var_i_name, var_j_name are the names of the variables.
        * check_func is a function that takes four arguments value_i and
        value_j. var_name_i, var_name_j
        Example. lambda i,j,name_i,name_j: i < j and returns true if
        the values passes the constraint, false otherwise.
        * description is a string descriptor of the constraint (helpful
        to determine what constraints triggered a search failure.
        """
        self.var_i_name = var_i_name
        self.var_j_name = var_j_name
        self.check_func = check_func
        self.description = description

    def get_variable_i_name(self):
        return self.var_i_name

    def get_variable_j_name(self):
        return self.var_j_name
    
    def check(self, state, value_i=None, value_j=None):
        """
        state is the csp state and should be an instance of
        CSPState.
        value_i and value_j are the values assigned to variable
        i and j respectively.   If they are not provided, the they are
        fetched from the state by looking up variable_i and variable_j's
        names.
        """
        variable_i = state.get_variable_by_name(self.var_i_name)
        if value_i is None and variable_i is not None:
            value_i = variable_i.get_assigned_value()

        variable_j = state.get_variable_by_name(self.var_j_name)            
        if value_j is None and variable_j is not None:
            value_j = variable_j.get_assigned_value()
            
        if value_i is not None and value_j is not None:
            return self.check_func(value_i, value_j,
                                   self.var_i_name, self.var_j_name)
        else:
            raise Exception("neither value_i nor value_j are set")

        # if values of i or j are not set, we really can't check
        # this constraint.  So the check passes.
        return True

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        name = "BinaryConstraint(%s, %s)" %(self.get_variable_i_name(),
                                            self.get_variable_j_name())
        if self.description is not None:
            name += " : %s" %(self.description)
        return name
        
class CSPState:
    """
    Representation of a single state in the CSP search tree.  One can
    think of this as the encapsulation of the Variable-domain (VD) table.
    """
    def __init__(self,
                 constraint_map,
                 variable_map,
                 variable_order,
                 variable_index):
        """
        constraint_map - a dictionary of variable names to
                         lists of associated constraints
        variable_map - a dictionary of variable names to
                       variable objects
        variable_order - the ordering in which variables are assigned
                       values are the names of variables
        variable_index - the position into the variable_order in which
                       we are currently making an assignment.
        """
        self.constraint_map = constraint_map
        self.variable_map = variable_map
        self.variable_order = variable_order
        self.variable_index = variable_index

    def copy(self):
        """
        Make a complete deep copy of this state; this should be
        done so that modifications to the VD table is only transmitted
        to children but not siblings (in the search tree).
        """
        # make a deep copy of the variable map.
        new_variable_map = {}
        for var_name, variable in self.variable_map.items():
            new_variable_map[var_name] = variable.copy()
        new_state = CSPState(self.constraint_map,
                             new_variable_map,
                             self.variable_order,
                             self.variable_index)
        return new_state

    def get_constraints_by_name(self, variable_name):
        """
        List only constraints associated with variable_name
        (where variable_name is variable_i in the constraint)
        """
        constraints = []
        for key, val in self.constraint_map.items():
            v_i, v_j = key
            if v_i == variable_name:
                constraints += val
        return constraints
    
    def get_all_constraints(self):
        """
        List all the constraints in this problem
        """
        constraints = []
        for key, val in self.constraint_map.items():
            constraints += val
        return constraints

    def get_all_variables(self):
        """
        List all the variable objects in this problem
        """
        variables = []
        for name in self.variable_order:
            variables.append(self.variable_map[name])
        return variables
        
    def get_current_variable_name(self):
        """
        Get the name of the variable currently being assigned.
        This function will returns None when in the root/initial state.
        """
        if self.variable_index >= 0:
            return self.get_variable_by_index(self.variable_index).get_name()
        else:
            return None

    def get_current_variable(self):
        """
        Get variable (object) currently being assigned.
        """
        if self.variable_index >= 0:
            return self.get_variable_by_index(self.variable_index)
        else:
            return None
        
    def set_variable_by_index(self, variable_index, variable_value):
        """
        assign variable (given index) the variable_value
        """
        variable = self.get_variable_by_index(variable_index)
        if variable is not None:
            variable.set_value(variable_value)
            self.variable_index = variable_index
        
    def get_variable_by_index(self, index):
        """
        fetch the index(th) variable object
        """
        if index >= 0 and index < len(self.variable_order):
            return self.variable_map[self.variable_order[index]]
        return None

    def get_variable_by_name(self, name):
        """
        fetch a variable object by the variable name
        """
        if name in self.variable_map:
            return self.variable_map[name]
        return None

    def is_solution(self):
        """
        Check if this csp state is a solution.
        Note we assume that constraint checking has been done
        on this state.  This merely checks if all the variables
        have an assignment
        """
        for var in self.variable_map.values():
            if not var.is_assigned():
                return False
        return True

    def solution(self):
        """
        return the set of tuples (var-name, var-value) for
        all the assigned variables
        """
        assignment = []
        for varname in self.variable_order:
            vnode = self.get_variable_by_name(varname)
            if vnode.is_assigned():
                assignment.append((vnode.get_name(),
                    vnode.get_assigned_value()))
        return assignment
    
    def __str__(self):
        return self.vd_table()
    
    def vd_table(self):
        """
        Output the vd table as a string for debugging.
        """
        buf = ""
        for var_name in self.variable_order:
            var = self.variable_map[var_name]
            if var.is_assigned():
                buf += "%s | %s*\n" %(var.get_name(),
                      var.get_assigned_value())
            else:
                buf += "%s | %s\n" %(var.get_name(),
                                     var.get_domain())
        return buf


def basic_constraint_checker(state, verbose=False):
    """
    Basic constraint checker used to check at every assignment
    whether the assignment passes all the constraints
    """
    constraints = state.get_all_constraints()
    for constraint in constraints:
        var_i = state.get_variable_by_name(constraint.get_variable_i_name())
        var_j = state.get_variable_by_name(constraint.get_variable_j_name())

        if not var_i.is_assigned() or not var_j.is_assigned():
            continue

        if not constraint.check(state):
            if verbose:
                print("CONSTRAINT-FAILS: %s" %(constraint))
            return False
    return True

class CSP:
    """
    Top-level wrapper object that encapsulates all the
    variables and constraints of a CSP problem
    """
    def __init__(self, constraints, variables):
        # Step 1: generate a constraint map, a mapping of pairs of
        # variable names to defined constraints on that pair.
        self.constraint_map = {}
        for constraint in constraints:
            i = constraint.get_variable_i_name()
            j = constraint.get_variable_j_name()
            tup = (i, j)
            if tup not in self.constraint_map:
                lst = []
                self.constraint_map[tup] = lst
            else:
                lst = self.constraint_map[tup]
            lst.append(constraint)

        # Step 2: generate the variable map, 
        self.variable_map = {}
        self.variable_order = []
        for var in variables:
            self.variable_map[var.get_name()] = var
            self.variable_order.append(var.get_name())

    def initial_state(self):
        """
        Returns the starting state of the CSP with no variables assigned.
        """
        return CSPState(self.constraint_map, self.variable_map,
                        self.variable_order, -1)

    def solve(self,
              constraint_checker=basic_constraint_checker,
              verbose=False):
        """
        Perform a depth-first search with backtracking to solve
        This CSP problem.

        The constraint_checker is a function that performs constraint-checking
        propagation on a CSPState.  By default the checker does
        basic constraint checking (without propagation).
        
        returns the solution state, and the search tree.
        """
        initial_state = self.initial_state()
        search_root = Node("ROOT", initial_state)
        agenda = [search_root]

        step = 0
        while len(agenda) > 0:
            cur_node = agenda.pop(0)
            state = cur_node.value        
            cur_node.step = step
            
            if verbose:
                print("-"*20)
                print("%d. EXAMINING:\n%s" %(step, state.vd_table()))
        
            if not constraint_checker(state, verbose):
                if verbose:
                    print("%d. FAIL:\n%s" %(step, state.vd_table()))
                cur_node.status = Node.FAILED
                step += 1
                continue

            if state.is_solution():
                cur_node.status = Node.SOLUTION
                if verbose:
                    print("%d. SOLUTION:\n%s" %(step, state.vd_table()))
                return state, search_root

            cur_node.status = Node.CONTINUE
            if verbose:
                print("%d. CONTINUE:\n%s" %(step, state.vd_table()))
        

            next_variable_index = state.variable_index + 1
            next_variable = state.get_variable_by_index(next_variable_index)
            values = next_variable.get_domain()

            children = []
            for value in values:
                new_state = state.copy()
                new_state.set_variable_by_index(next_variable_index, value)
                children.append(Node(str(value), new_state))

            cur_node.add_children(children)
            agenda = children + agenda
            step += 1
            
        # fail! no solution
        return None, search_root
    
class Node:
    """
    A tree node that csp.solve() uses/returns that keeps track of the CSP
    search tree.
    """
    UNEXTENDED = "u"
    FAILED = "f"
    CONTINUE = "c"
    SOLUTION = "*"
    
    def __init__(self, label, value):
        self.label = label
        self.status = Node.UNEXTENDED
        self.value = value
        self.step = '-'
        self.children = []

    def add_children(self, children):
        self.children += children
        
    def __str__(self):
        return self.label
            
    def tree_to_string(self, node, depth=0):
        pad = depth*4*" "
        current_var = node.value.get_current_variable_name()    
        if current_var is not None:
            buf = "%s%s=%s(%s,%s)\n" %(pad,
                                       current_var,
                                       node.label,
                                       node.status,
                                       node.step)
        else:
            buf = "%s%s\n" %(pad, node.label)
            
        for child in node.children:
            buf += self.tree_to_string(child, depth+1)
        return buf

    
def simple_csp_problem():
    """
    Formulation of a simple CSP problem that attempts to find
    an assignment to 4 variables: A,B,C,D.  With the constraint that
    A < B < C < D.
    """
    variables = []
    domain = [1, 2, 3, 4]
    
    variables.append(Variable("A", domain))
    variables.append(Variable("B", domain))
    variables.append(Variable("C", domain))
    variables.append(Variable("D", domain))

    constraints = []

    def less_than(val_a, val_b, name_a=None, name_b=None):
        return val_a < val_b
    
    constraints.append(BinaryConstraint("A", "B", less_than, "A < B"))
    constraints.append(BinaryConstraint("B", "C", less_than, "B < C"))
    constraints.append(BinaryConstraint("C", "D", less_than, "C < D"))

    def not_equal(val_a, val_b, name_a=None, name_b=None):
        return val_a != val_b
    
    constraints.append(BinaryConstraint("A", "B", not_equal, "A != B"))
    constraints.append(BinaryConstraint("B", "C", not_equal, "B != C"))
    constraints.append(BinaryConstraint("C", "D", not_equal, "C != D"))
    constraints.append(BinaryConstraint("A", "D", not_equal, "A != D"))
    return CSP(constraints, variables)

def solve_csp_problem(problem, checker, verbose=False):
    """
    problem is a function that returns a CSP object that we can solve.
    checker is a function that implements the contraint checking.
    variable_order_cmp is a comparator for ordering the variables.
    """
    csp = problem()
    answer, search_tree = csp.solve(checker, verbose=verbose)

    if verbose:
        if answer is not None:
            print("ANSWER: %s" %(answer.solution()))
        else:
            print("NO SOLUTION FOUND")
        if search_tree is not None:
            print("TREE:\n")
            print(search_tree.tree_to_string(search_tree))
        
    return answer, search_tree
        
if __name__ == "__main__":
    checker = basic_constraint_checker
    #fc_checker = forward_checking    
    #fcps_checker = forward_checking_prop_singleton
    solve_csp_problem(simple_csp_problem, checker, verbose=True)


--------------------
0. EXAMINING:
A | [1, 2, 3, 4]
B | [1, 2, 3, 4]
C | [1, 2, 3, 4]
D | [1, 2, 3, 4]

0. CONTINUE:
A | [1, 2, 3, 4]
B | [1, 2, 3, 4]
C | [1, 2, 3, 4]
D | [1, 2, 3, 4]

--------------------
1. EXAMINING:
A | 1*
B | [1, 2, 3, 4]
C | [1, 2, 3, 4]
D | [1, 2, 3, 4]

1. CONTINUE:
A | 1*
B | [1, 2, 3, 4]
C | [1, 2, 3, 4]
D | [1, 2, 3, 4]

--------------------
2. EXAMINING:
A | 1*
B | 1*
C | [1, 2, 3, 4]
D | [1, 2, 3, 4]

CONSTRAINT-FAILS: BinaryConstraint(A, B) : A < B
2. FAIL:
A | 1*
B | 1*
C | [1, 2, 3, 4]
D | [1, 2, 3, 4]

--------------------
3. EXAMINING:
A | 1*
B | 2*
C | [1, 2, 3, 4]
D | [1, 2, 3, 4]

3. CONTINUE:
A | 1*
B | 2*
C | [1, 2, 3, 4]
D | [1, 2, 3, 4]

--------------------
4. EXAMINING:
A | 1*
B | 2*
C | 1*
D | [1, 2, 3, 4]

CONSTRAINT-FAILS: BinaryConstraint(B, C) : B < C
4. FAIL:
A | 1*
B | 2*
C | 1*
D | [1, 2, 3, 4]

--------------------
5. EXAMINING:
A | 1*
B | 2*
C | 2*
D | [1, 2, 3, 4]

CONSTRAINT-FAILS: BinaryConstraint(B, C) : B < C
5. FAIL:
A | 1*
B

# Data Reader

In [ ]:
"""
A set of utility functions for reading in the data format used by Keith T.
Poole on voteview.com.

You can download additional data that will work with these functions, for
any Congress going back to the 1st, on that site.
"""
import csv
from copy import deepcopy

def legislator_info(legislator):
    district = ''
    if legislator['district'] > 0: district = '-%s' % legislator['district']
    return "%s (%s%s)" % (legislator['name'], legislator['state'], district)

def vote_info(vote):
    if not vote['name']: return vote['number']
    return "%s: %s" % (vote['number'], vote['name'])

def is_interesting(vote):
    return (vote['name'] != '')

def title_case(str):
    chars = list(str)
    chars[0] = chars[0].upper()
    for i in range(1, len(chars)):
        if chars[i-1] not in ' -': chars[i] = chars[i].lower()
    return ''.join(chars)

state_codes = {}
f = open('states.dat')
for line in f:
    state_codes[int(line[0:2])] = title_case(line[6:].strip())
f.close()

party_codes = {}
f = open('party3.dat')
for line in f:
    party_codes[int(line[2:6])] = line[8:].strip()
f.close()

def vote_meaning(n):
    if n in [1, 2, 3]: return 1
    elif n in [4, 5, 6]: return -1
    else: return 0

def read_congress_data(filename):
    """
    Reads a database of Congressional information in the format that comes
    from Keith T. Poole's voteview.com.
    """
    f = open(filename)
    legislators = []
    for line in f:
        line = line.rstrip()
        person = {}
        person['state'] = state_codes[int(line[8:10])]
        person['district'] = int(line[10:12])
        person['party'] = party_codes[int(line[19:23])]
        name = line[25:36].strip()
        person['name'] = title_case(name.replace("  ", ", "))
        person['votes'] = [vote_meaning(int(x)) for x in line[36:]]
        legislators.append(person)
    f.close()
    return legislators

def read_vote_data(filename):
    """
    Reads a CSV file of data on the votes that were taken.
    """
    f = open(filename, encoding='utf-8')
    csv_reader = csv.reader(f)
    votes = []
    for row in csv_reader:
        vote = {}
        vote['date'] = row[0]
        vote['number'] = row[3]
        vote['motion'] = row[4]
        vote['name'] = row[6]
        vote['result'] = row[5]
        votes.append(vote)
    f.close()
    return votes

def limit_votes(legislators, votes, n):
    indices = [i for i in range(len(legislators[0]['votes'])-1, -1, -1) if
    is_interesting(votes[i])][:n]

    newleg = []
    for leg in legislators:
        leg = deepcopy(leg)
        leg['votes'] = [leg['votes'][i] for i in indices]
        found_any_votes = False
        for vote in leg['votes']:
            if vote != 0:
                found_any_votes = True
                break
        if found_any_votes: newleg.append(leg)
    newvotes = [votes[i] for i in indices]
    return newleg, newvotes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Classify

In [ ]:
try:
    set()
except:
    from sets import Set as set

import math
import random

INFINITY = 1e100
def crosscheck_groups(people):
    # Split up the data in an interesting way.
    group1 = people[0::4] + people[3::4]
    group2 = people[1::4] + people[2::4]
    return group1, group2

def random_split_groups(people):
    # Split up the data randomly, into two groups of ~equal size
    group1 = random.sample(people, len(people)/2)
    group2 = [ x for x in people if not x in group1 ]
    return (group1, group2)
    
def evaluate(factory, group1, group2, verbose=0):
    score = 0
    for (test, train) in ((group1, group2), (group2, group1)):
        classifier = factory(train)
        for legislator in test:
            gold_standard = legislator['party']
            predicted = classifier(legislator)
            if gold_standard == predicted:
                score += 1
                if verbose >= 2:
                    print("%s: %s (correct)" % (legislator_info(legislator),),predicted)
            else:
                if verbose >= 1:
                    print("* %s: got %s, actually %s" %\
                    (legislator_info(legislator), predicted, gold_standard))
    if verbose >= 1:
        print("Accuracy: %d/%d" % (score, len(group1) + len(group2)))
    return score

def hamming_distance(list1, list2):
    """ Calculate the Hamming distance between two lists """
    # Make sure we're working with lists
    # Sorry, no other iterables are permitted
    assert isinstance(list1, list)
    assert isinstance(list2, list)

    dist = 0

    # 'zip' is a Python builtin, documented at
    # <http://www.python.org/doc/lib/built-in-funcs.html>
    for item1, item2 in zip(list1, list2):
        if item1 != item2: dist += 1
    return dist

edit_distance = hamming_distance

def nearest_neighbors(distance, k=1):
    def nearest_neighbors_classifier(train):
        def classify_value(query):
            best_distance = INFINITY
            ordered = sorted(train, key=lambda x: distance(query['votes'],
            x['votes']))
            nearest = [x['party'] for x in ordered[:k]]
            best_class = None
            best_count = 0
            for party in nearest:
                count = nearest.count(party)
                if count > best_count:
                    best_count = count
                    best_class = party
            return best_class
        return classify_value
    return nearest_neighbors_classifier

def homogeneous_disorder(yes, no):
    result = 0
    if homogeneous_value(yes): result -= len(yes)
    if homogeneous_value(no): result -= len(no)
    return result

def partition(legislators, vote_index, vote_value):
    # Find the people who voted a particular way, and the people who didn't.
    # Yes, No, and Abstain/Absent count as three different options here.
    matched = []
    unmatched = []
    for leg in legislators:
        if leg['votes'][vote_index] == vote_value:
            matched.append(leg)
        else:
            unmatched.append(leg)
    return matched, unmatched

def homogeneous_value(lst):
    """If this list contains just a single value, return it."""
    assert isinstance(lst[0], str)
    for item in lst[1:]:
        if item != lst[0]: return None
    return lst[0]

class CongressIDTree(object):
    def __init__(self, legislators, vote_meanings, disorder_func=None):
        if disorder_func is None: disorder_func = homogeneous_disorder
        self.vote_meanings = vote_meanings
        homog_test = homogeneous_value([leg['party'] for leg in legislators])
        if homog_test:
            self.leaf_value = homog_test
        else:
            self.leaf_value = None
            best_disorder = INFINITY
            best_criterion = None
            for vote_index in range(len(legislators[0]['votes'])):
                for vote_value in [1, 0, -1]:
                    yes, no = partition(legislators, vote_index, vote_value)
                    if len(yes) == 0 or len(no) == 0: continue
                    disord = disorder_func([y['party'] for y in yes],
                                           [n['party'] for n in no])
                    if disord < best_disorder:
                        best_disorder = disord
                        best_criterion = (vote_index, vote_value)

            if best_criterion is None:
                # No reasonable criteria left, so give up
                self.leaf_value = 'Unknown'
                return
            vote_index, vote_value = best_criterion
            self.criterion = best_criterion
            self.disorder = best_disorder
            yes_class, no_class = partition(legislators, vote_index, vote_value)
            yes_values = [y['party'] for y in yes]
            no_values = [n['party'] for n in no]
            self.yes_branch = CongressIDTree(yes_class, vote_meanings,
            disorder_func)
            self.no_branch = CongressIDTree(no_class, vote_meanings,
            disorder_func)
    
    def classify(self, legislator):
        if self.leaf_value: return self.leaf_value
        vote_index, vote_value = self.criterion
        if legislator['votes'][vote_index] == vote_value:
            return self.yes_branch.classify(legislator)
        else:
            return self.no_branch.classify(legislator)

    def __str__(self):
        return '\n'+self._str(0)

    def _str(self, indent):
        if self.leaf_value:
            return str(self.leaf_value)
        
        vote_index, vote_value = self.criterion
        value_name = 'Abstain/Absent'
        if vote_value == -1: value_name = 'No'
        elif vote_value == 1: value_name = 'Yes'
        vote_name = vote_info(self.vote_meanings[vote_index])
        indentation = ' '*indent
        disord_string = 'Disorder: %s' % self.disorder
        yes_string = indentation+'+ '+self.yes_branch._str(indent+2)
        no_string = indentation+'- '+self.no_branch._str(indent+2)
        return ("%(disord_string)s\n%(indentation)s%(value_name)s on %(vote_name)s:"
                "\n%(yes_string)s\n%(no_string)s") % locals()


def idtree_maker(vote_meanings, disorder_func):
    def train_classifier(train):
        idtree = CongressIDTree(train, vote_meanings, disorder_func)
        def classify_value(query):
            return idtree.classify(query)
        return classify_value
    return train_classifier


# **To be implemented code**

In [ ]:
import math

##
## CSP portion of the laboratory.
##

# Implement basic forward checking on the CSPState see csp.py
def forward_checking(state, verbose=False):
    # Before running Forward checking we must ensure
    # that constraints are okay for this state.
    basic = basic_constraint_checker(state, verbose)
    if not basic:
        return False

    X = state.get_current_variable()
    x = None

    if X == None:
      return True

    x = X.get_assigned_value()
   
    for constraint in state.get_constraints_by_name(X.get_name()):
      if constraint.get_variable_i_name() != X.get_name():
        Y = state.get_variable_by_name(constraint.get_variable_i_name())
      else:
        Y = state.get_variable_by_name(constraint.get_variable_j_name())
      
      for y in Y.get_domain():

        if not constraint.check(state,x,y):
          Y.reduce_domain(y)

        if not len(Y.get_domain()):
          return False 
    return True


# Now Implement forward checking + (constraint) propagation through
# singleton domains.
def forward_checking_prop_singleton(state, verbose=False):
    # Run forward checking first.
    fc_checker = forward_checking(state, verbose)
    if not fc_checker:
        return False

    variables = state.get_all_variables()
    var_queue = []
    
    for var in variables: 
      if var.domain_size() == 1:
         var_queue.append(var)

    while var_queue:
      X = var_queue.pop(0)
      x = X.get_domain()[0] 

      for constraint in state.get_constraints_by_name(X.get_name()):
        if constraint.get_variable_i_name() != X.get_name():
           Y = state.get_variable_by_name(constraint.get_variable_i_name())
        else:
           Y = state.get_variable_by_name(constraint.get_variable_j_name())
        
        for y in Y.get_domain():
          if not constraint.check(state,x,y):
             Y.reduce_domain(y)
          if not len(Y.get_domain()):
            return False 
    return True   

## The code here are for the tester
## Do not change.

def csp_solver_tree(problem, checker):
    problem_func = globals()[problem]
    checker_func = globals()[checker]
    answer, search_tree = problem_func().solve(checker_func)
    return search_tree.tree_to_string(search_tree)

##
## CODE for the learning portion of lab 4.
##

### Data sets for the lab
## You will be classifying data from these sets.
senate_people = read_congress_data('S110.ord')
senate_votes = read_vote_data('S110desc.csv')

house_people = read_congress_data('H110.ord')
house_votes = read_vote_data('H110desc.csv')

last_senate_people = read_congress_data('S109.ord')
last_senate_votes = read_vote_data('S109desc.csv')


### Part 1: Nearest Neighbors
## An example of evaluating a nearest-neighbors classifier.
senate_group1, senate_group2 = crosscheck_groups(senate_people)
#evaluate(nearest_neighbors(hamming_distance, 1), senate_group1, senate_group2, verbose=1)

## Write the euclidean_distance function.
## This function should take two lists of integers and
## find the Euclidean distance between them.
## See 'hamming_distance()' in classify.py for an example that
## computes Hamming distances.

def euclidean_distance(list1, list2):
    # this is not the right solution!
    assert isinstance(list1, list)
    assert isinstance(list2, list)

    dist = 0

    for item1, item2 in zip(list1, list2): 
      dist += (item1 - item2)**2

    return math.sqrt(dist)
    

#Once you have implemented euclidean_distance, you can check the results:
evaluate(nearest_neighbors(euclidean_distance, 1), senate_group1, senate_group2)

## By changing the parameters you used, you can get a classifier factory that
## deals better with independents. Make a classifier that makes at most 3
## errors on the Senate.

my_classifier = nearest_neighbors(hamming_distance, 1)
#evaluate(my_classifier, senate_group1, senate_group2, verbose=1)

### Part 2: ID Trees
#print(CongressIDTree(senate_people, senate_votes, homogeneous_disorder))

## Now write an information_disorder function to replace homogeneous_disorder,
## which should lead to simpler trees.

def information_disorder(yes, no):
     
     total_yes = float(len(yes))
     total_no = float(len(no))
     total = total_yes + total_no

     log_yes = 0
     log_no = 0

     if homogeneous_value(yes):
       log_yes = 0
     else:
       dict_yes = {}
        
       for data in yes:
           if data in dict_yes:
             dict_yes[data] += 1
           else:
             dict_yes[data] = 1
       log_yes = 0      

       for data, nr in dict_yes.items():
         fraction_yes = float(nr)/total_yes
         log_yes += -fraction_yes*math.log(fraction_yes,2)

     if homogeneous_value(no):
       log_no = 0
     else:
       
       dict_no = {}

       for data in no:
         if data in dict_no:
           dict_no[data] += 1
         else:
           dict_no[data] = 1
       
       log_no = 0

       for data, nr in dict_no.items():
         fraction_no = float(nr)/total_no
         log_no += -fraction_no*math.log(fraction_no,2)
     
     return (total_yes/total)*log_yes + (total_no/total)*log_no 
      



#print(CongressIDTree(senate_people, senate_votes, information_disorder))
#evaluate(idtree_maker(senate_votes, homogeneous_disorder), senate_group1, senate_group2)

## Now try it on the House of Representatives. However, do it over a data set
## that only includes the most recent n votes, to show that it is possible to
## classify politicians without ludicrous amounts of information.

def limited_house_classifier(house_people, house_votes, n, verbose = False):
    house_limited, house_limited_votes = limit_votes(house_people,
    house_votes, n)
    house_limited_group1, house_limited_group2 = crosscheck_groups(house_limited)

    if verbose:
        print("ID tree for first group:")
        print(CongressIDTree(house_limited_group1, house_limited_votes,
                             information_disorder))
        print()
        print("ID tree for second group:")
        print(CongressIDTree(house_limited_group2, house_limited_votes,
                             information_disorder))
        print()
        
    return evaluate(idtree_maker(house_limited_votes, information_disorder),
                    house_limited_group1, house_limited_group2)

                                   
## Find a value of n that classifies at least 430 representatives correctly.
## Hint: It's not 10.
N_1 = 50
rep_classified = limited_house_classifier(house_people, house_votes, N_1)

## Find a value of n that classifies at least 90 senators correctly.
N_2 = 70
senator_classified = limited_house_classifier(senate_people, senate_votes, N_2)

## Now, find a value of n that classifies at least 95 of last year's senators correctly.
N_3 = 50
old_senator_classified = limited_house_classifier(last_senate_people, last_senate_votes, N_3)


## This function is used by the tester, please don't modify it!
def eval_test(eval_fn, group1, group2, verbose = 0):
    """ Find eval_fn in globals(), then execute evaluate() on it """
    # Only allow known-safe eval_fn's
    if eval_fn in [ 'my_classifier' ]:
        return evaluate(globals()[eval_fn], group1, group2, verbose)
    else:
        raise Exception("Error: Tester tried to use an invalid evaluation function: '%s'" % eval_fn)

# Moose CSP

In [ ]:
"""
Implementation of the Moose/Obama/Biden/McCain/Palin problem.
"""
import sys

def moose_csp_problem():
    constraints = []

    # We start with the reduced domain.
    # So the constraint that McCain must sit in seat 1 is already
    # covered.
    variables = []
    variables.append(Variable("1", ["Mc"]))
    variables.append(Variable("2", ["Y", "M", "P"]))
    variables.append(Variable("3", ["Y", "M", "O", "B"]))
    variables.append(Variable("4", ["Y", "M", "O", "B"]))
    variables.append(Variable("5", ["Y", "M", "O", "B"]))
    variables.append(Variable("6", ["Y", "M", "P"]))

    # these are all variable pairing of adjacent seats
    adjacent_pairs = [("1", "2"), ("2", "1"),
                      ("2", "3"), ("3", "2"),
                      ("3", "4"), ("4", "3"),
                      ("4", "5"), ("5", "4"),
                      ("5", "6"), ("6", "5"),
                      ("6", "1"), ("1", "6")]
    # now we construct the set of non-adjacent seat pairs.
    nonadjacent_pairs = []
    variable_names = ["1", "2", "3", "4", "5", "6"]
    for x in range(len(variable_names)):
        for y in range(x,len(variable_names)):
            if x == y:
                continue
            tup = (variable_names[x], variable_names[y])
            rev = (variable_names[y], variable_names[x])
            if tup not in adjacent_pairs:
                nonadjacent_pairs.append(tup)
            if rev not in adjacent_pairs:
                nonadjacent_pairs.append(rev)

    # all pairs is the set of all distinct seating pairs
    # this list is useful for checking where
    # the two seat are assigned to the same person.
    all_pairs = adjacent_pairs + nonadjacent_pairs

    # 1. The Moose is afraid of Palin
    def M_not_next_to_P(val_a, val_b, name_a, name_b):
        if (val_a == "M" and val_b == "P") or (val_a == "P" and val_b == "M"):
            return False
        return True
    for pair in adjacent_pairs:
        constraints.append(BinaryConstraint(pair[0], pair[1], M_not_next_to_P,
                                            "Moose can't be next to Palin"))

    # 2. Obama and Biden must sit next to each other.
    # This constraint can be directly phrased as:
    #
    #   for all sets of adjacents seats
    #      there must exist one pair where O & B are assigned
    #
    #   C(1,2) or C(2,3) or C(3,4) or ... or C(6,1)
    #
    # where C is a binary constraint that checks
    # whether the value of the two variables have values O and B
    #
    # However the way our checker works, the constraint needs to be
    # expressed as a big AND.
    # So that when any one of the binary constraints
    # fails the entire assignment fails.
    #
    # To turn our original OR formulation to an AND:
    # We invert the constraint condition as:
    #
    #  for all sets of nonadjacent seats
    #     there must *not* exist a pair where O & B are assigned.
    #
    #  not C(1,3) and not C(1,4) and not C(1,5) ... not C(6,4)
    #
    # Here C checks whether the values assigned are O and B.
    #
    # Finally, this is an AND of all the binary constraints as required.

    def OB_not_next_to_each_other(val_a, val_b, name_a, name_b):
        if (val_a == "O" and val_b == "B") or \
                (val_a == "B" and val_b == "O"):
            return False
        return True
    
    for pair in nonadjacent_pairs:
        constraints.append(
            BinaryConstraint(pair[0], pair[1],
                             OB_not_next_to_each_other,
                             "Obama, Biden must be next to each-other"))

    # 3. McCain and Palin must sit next to each other
    def McP_not_next_to_each_other(val_a, val_b, name_a, name_b):
        if (val_a == "P" and val_b == "Mc") or (val_a == "Mc" and val_b == "P"):
            return False
        return True

    for pair in nonadjacent_pairs:
        constraints.append(
            BinaryConstraint(pair[0], pair[1],
                             McP_not_next_to_each_other,
                             "McCain and Palin must be next to each other"))

    # 4. Obama + Biden can't sit next to Palin or McCain
    def OB_not_next_to_McP(val_a, val_b, name_a, name_b):
        if ((val_a == "O" or val_a == "B") \
                and (val_b == "Mc" or val_b == "P")) or \
                ((val_b == "O" or val_b == "B") \
                     and (val_a == "Mc" or val_a == "P")):
            return False
        return True
    for pair in adjacent_pairs:
        constraints.append(
            BinaryConstraint(pair[0], pair[1],
                             OB_not_next_to_McP,
                             "McCain, Palin can't be next to Obama, Biden"))

    # No two seats can be occupied by the same person
    def not_same_person(val_a, val_b, name_a, name_b):
        return val_a != val_b
    for pair in all_pairs:
        constraints.append(
            BinaryConstraint(pair[0], pair[1],
                             not_same_person,
                             "No two seats can be occupied by the same person"))
    return CSP(constraints, variables)

checker_type = "dfs" #change here
        
if checker_type == "dfs":
    checker = basic_constraint_checker
elif checker_type == "fc":
    checker = forward_checking
elif checker_type == "fcps":
    checker = lab4.forward_checking_prop_singleton
else:
    checker = basic_constraint_checker

solve_csp_problem(moose_csp_problem, checker, verbose=True)


--------------------
0. EXAMINING:
1 | ['Mc']
2 | ['Y', 'M', 'P']
3 | ['Y', 'M', 'O', 'B']
4 | ['Y', 'M', 'O', 'B']
5 | ['Y', 'M', 'O', 'B']
6 | ['Y', 'M', 'P']

0. CONTINUE:
1 | ['Mc']
2 | ['Y', 'M', 'P']
3 | ['Y', 'M', 'O', 'B']
4 | ['Y', 'M', 'O', 'B']
5 | ['Y', 'M', 'O', 'B']
6 | ['Y', 'M', 'P']

--------------------
1. EXAMINING:
1 | Mc*
2 | ['Y', 'M', 'P']
3 | ['Y', 'M', 'O', 'B']
4 | ['Y', 'M', 'O', 'B']
5 | ['Y', 'M', 'O', 'B']
6 | ['Y', 'M', 'P']

1. CONTINUE:
1 | Mc*
2 | ['Y', 'M', 'P']
3 | ['Y', 'M', 'O', 'B']
4 | ['Y', 'M', 'O', 'B']
5 | ['Y', 'M', 'O', 'B']
6 | ['Y', 'M', 'P']

--------------------
2. EXAMINING:
1 | Mc*
2 | Y*
3 | ['Y', 'M', 'O', 'B']
4 | ['Y', 'M', 'O', 'B']
5 | ['Y', 'M', 'O', 'B']
6 | ['Y', 'M', 'P']

2. CONTINUE:
1 | Mc*
2 | Y*
3 | ['Y', 'M', 'O', 'B']
4 | ['Y', 'M', 'O', 'B']
5 | ['Y', 'M', 'O', 'B']
6 | ['Y', 'M', 'P']

--------------------
3. EXAMINING:
1 | Mc*
2 | Y*
3 | Y*
4 | ['Y', 'M', 'O', 'B']
5 | ['Y', 'M', 'O', 'B']
6 | ['Y', 'M', 'P']

CONS

(<__main__.CSPState at 0x7f4f3e0a4f60>, <__main__.Node at 0x7f4f3ddd55c0>)

# Map coloring

In [ ]:
"""
Implementation of the Map coloring problem.
"""
import sys

def map_coloring_csp_problem():
    constraints = []

    variables = []
    # order of the variables here is the order given in the problem
    variables.append(Variable("MA", ["B"]))
    variables.append(Variable("TX", ["R"]))
    variables.append(Variable("NE", ["R", "B", "Y"]))
    variables.append(Variable("OV", ["R", "B", "Y"]))
    variables.append(Variable("SE", ["R", "B", "Y"]))
    variables.append(Variable("GL", ["R", "B", "Y"]))
    variables.append(Variable("MID",["R", "B", "Y"]))
    variables.append(Variable("MW", ["R", "B", "Y"]))
    variables.append(Variable("SO", ["R", "B"]))
    variables.append(Variable("NY", ["R", "B"]))
    variables.append(Variable("FL", ["R", "B"]))

    # these are all variable pairing of adjacent seats
    edges = [("NE", "NY"),
             ("NE", "MA"),
             ("MA", "NY"),
             ("GL", "NY"),
             ("GL", "OV"),
             ("MID", "NY"),
             ("OV", "NY"),
             ("OV", "MID"),
             ("MW", "OV"),
             ("MW", "TX"),
             ("TX", "SO"),
             ("SO", "OV"),
             ("SO", "FL"),
             ("FL", "SE"),
             ("SE", "MID"),
             ("SE", "SO")]
    # duplicate the edges the other way.
    all_edges = []
    for edge in edges:
        all_edges.append((edge[0], edge[1]))
        all_edges.append((edge[1], edge[0]))
        
    forbidden = [("R", "B"), ("B", "R"), ("Y", "Y")]
    
    # not allowed constraints:
    def forbidden_edge(val_a, val_b, name_a, name_b):
        if (val_a, val_b) in forbidden or (val_b, val_a) in forbidden:
            return False
        return True

    for pair in all_edges:
        constraints.append(
            BinaryConstraint(pair[0], pair[1],
                             forbidden_edge,
                             "R-B, B-R, Y-Y edges are not allowed"))

    return CSP(constraints, variables)

checker_type = "dfs" #change here
        
if checker_type == "dfs":
    checker = basic_constraint_checker
elif checker_type == "fc":
    checker = forward_checking
elif checker_type == "fcps":
    checker = forward_checking_prop_singleton
else:
    checker = forward_checking_prop_singleton

solve_csp_problem(map_coloring_csp_problem, checker, verbose=True)


--------------------
0. EXAMINING:
MA | ['B']
TX | ['R']
NE | ['R', 'B', 'Y']
OV | ['R', 'B', 'Y']
SE | ['R', 'B', 'Y']
GL | ['R', 'B', 'Y']
MID | ['R', 'B', 'Y']
MW | ['R', 'B', 'Y']
SO | ['R', 'B']
NY | ['R', 'B']
FL | ['R', 'B']

0. CONTINUE:
MA | ['B']
TX | ['R']
NE | ['R', 'B', 'Y']
OV | ['R', 'B', 'Y']
SE | ['R', 'B', 'Y']
GL | ['R', 'B', 'Y']
MID | ['R', 'B', 'Y']
MW | ['R', 'B', 'Y']
SO | ['R', 'B']
NY | ['R', 'B']
FL | ['R', 'B']

--------------------
1. EXAMINING:
MA | B*
TX | ['R']
NE | ['R', 'B', 'Y']
OV | ['R', 'B', 'Y']
SE | ['R', 'B', 'Y']
GL | ['R', 'B', 'Y']
MID | ['R', 'B', 'Y']
MW | ['R', 'B', 'Y']
SO | ['R', 'B']
NY | ['R', 'B']
FL | ['R', 'B']

1. CONTINUE:
MA | B*
TX | ['R']
NE | ['R', 'B', 'Y']
OV | ['R', 'B', 'Y']
SE | ['R', 'B', 'Y']
GL | ['R', 'B', 'Y']
MID | ['R', 'B', 'Y']
MW | ['R', 'B', 'Y']
SO | ['R', 'B']
NY | ['R', 'B']
FL | ['R', 'B']

--------------------
2. EXAMINING:
MA | B*
TX | R*
NE | ['R', 'B', 'Y']
OV | ['R', 'B', 'Y']
SE | ['R', 'B', 'Y']
GL |

(<__main__.CSPState at 0x7f4f3da21e80>, <__main__.Node at 0x7f4f3dcb3198>)

# Tester

In [ ]:
from xmlrpc import client
import traceback
import sys
import os
import tarfile

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO


# This is a skeleton for what the tester should do. Ideally, this module
# would be imported in the pset and run as its main function. 

# We need the following rpc functions. (They generally take username and
# password, but you could adjust this for whatever security system.)
#
# tester.submit_code(username, password, pset, studentcode)
#   'pset' is a string such as 'ps0'. studentcode is a string containing
#   the contents of the corresponding file, ps0.py. This stores the code on
#   the server so we can check it later for cheating, and is a prerequisite
#   to the tester returning a grade.
#
# tester.get_tests(pset)
#   returns a list of tuples of the form (INDEX, TYPE, NAME, ARGS):
#     INDEX is a unique integer that identifies the test.
#     TYPE should be one of either 'VALUE' or 'FUNCTION'.
#     If TYPE is 'VALUE', ARGS is ignored, and NAME is the name of a
#     variable to return for this test.  The variable must be an attribute
#     of the lab module.
#     If TYPE is 'FUNCTION', NAME is the name of a function in the lab module
#     whose return value should be the answer to this test, and ARGS is a
#     tuple containing arguments for the function.
#
# tester.send_answer(username, password, pset, index, answer)
#   Sends <answer> as the answer to test case <index> (0-numbered) in the pset
#   named <pset>. Returns whether the answer was correct, and an expected
#   value.
#
# tester.status(username, password, pset)
#   A string that includes the official score for this user on this pset.
#   If a part is missing (like the code), it should say so.

# Because I haven't written anything on the server side, test_online has never
# been tested.

def test_summary(dispindex, ntests, testname):
    return "Test %d/%d (%s)" % (dispindex, ntests, testname)
  
tests = []

def show_result(testsummary, testcode, correct, got, expected, verbosity):
    """ Pretty-print test results """
    if correct:
        if verbosity > 0:
            print("%s: Correct." % testsummary)
        if verbosity > 1:
            print('\t', testcode)
            print()
    else:
        print("%s: Incorrect." % testsummary)
        print('\t', testcode)
        print("Got:     ", got)
        print("Expected:", expected)

def show_exception(testsummary, testcode):
    """ Pretty-print exceptions (including tracebacks) """
    print("%s: Error." % testsummary)
    print("While running the following test case:")
    print('\t', testcode)
    print("Your code encountered the following error:")
    traceback.print_exc()
    print()


def get_lab_module():
    # Try the easy way first
    try:
        from tests import lab_number
    except ImportError:
        lab_number = None
        
    if lab_number != None:
        lab = __import__('lab%s' % lab_number)
        return lab
        
    lab = None

    for labnum in range(10):
        try:
            lab = __import__('lab%s' % labnum)
        except ImportError:
            pass

    if lab == None:
        raise ImportError("Cannot find your lab; or, error importing it.  Try loading it by running 'python labN.py' (for the appropriate value of 'N').")

    if not hasattr(lab, "LAB_NUMBER"):
        lab.LAB_NUMBER = labnum
    
    return lab

def type_decode(arg, lab):
    """
    XMLRPC can only pass a very limited collection of types.
    Frequently, we want to pass a subclass of 'list' in as a test argument.
    We do that by converting the sub-type into a regular list of the form:
    [ 'TYPE', (data) ] (ie., AND(['x','y','z']) becomes ['AND','x','y','z']).
    This function assumes that TYPE is a valid attr of 'lab' and that TYPE's
    constructor takes a list as an argument; it uses that to reconstruct the
    original data type.
    """
    if isinstance(arg, list) and len(arg) >= 1: # We'll leave tuples reserved for some other future magic
        try:
            mytype = arg[0]
            data = arg[1:]
            return getattr(lab, mytype)([ type_decode(x, lab) for x in data ])
        except AttributeError:
            return [ type_decode(x, lab) for x in arg ]
        except TypeError:
            return [ type_decode(x, lab) for x in arg ]
    else:
        return arg

    
def type_encode(arg):
    """
    Encode trees as lists in a way that can be decoded by 'type_decode'
    """
    if isinstance(arg, list) and not type(arg) in (list,tuple):
        return [ arg.__class__.__name__ ] + [ type_encode(x) for x in arg ]
    elif hasattr(arg, '__class__') and arg.__class__.__name__ == 'IF':
        return [ 'IF', type_encode(arg._conditional), type_encode(arg._action), type_encode(arg._delete_clause) ]
    else:
        return arg

    
def run_test(test, lab):
    """
    Takes a 'test' tuple as provided by the online tester
    (or generated by the offline tester) and executes that test,
    returning whatever output is expected (the variable that's being
    queried, the output of the function being called, etc)

    'lab' (the argument) is the module containing the lab code.
    
    'test' tuples are in the following format:
      'id': A unique integer identifying the test
      'type': One of 'VALUE', 'FUNCTION', 'MULTIFUNCTION', or 'FUNCTION_ENCODED_ARGS'
      'attr_name': The name of the attribute in the 'lab' module
      'args': a list of the arguments to be passed to the function; [] if no args.
      For 'MULTIFUNCTION's, a list of lists of arguments to be passed in
    """
    id, mytype, attr_name, args = test

    attr = getattr(lab, attr_name)

    if mytype == 'VALUE':
        return attr
    elif mytype == 'FUNCTION':
        return apply(attr, args)
    elif mytype == 'MULTIFUNCTION':
        return [ run_test( (id, 'FUNCTION', attr_name, FN), lab) for FN in args ]
    elif mytype == 'FUNCTION_ENCODED_ARGS':
        return run_test( (id, 'FUNCTION', attr_name, type_decode(args, lab)), lab )
    else:
        raise Exception("Test Error: Unknown TYPE '%s'.  Please make sure you have downloaded the latest version of the tester script.  If you continue to see this error, contact a TA.")


def test_offline(verbosity=1):
    """ Run the unit tests in 'tests.py' """
#    import tests as tests_module
    
#    tests = [ (x[:-8],
#               getattr(tests_module, x),
#               getattr(tests_module, "%s_testanswer" % x[:-8]),
#               getattr(tests_module, "%s_expected" % x[:-8]),
#               "_".join(x[:-8].split('_')[:-1]))
#              for x in tests_module.__dict__.keys() if x[-8:] == "_getargs" ]

#    tests = tests_module.get_tests()
    global tests

    ntests = len(tests)
    ncorrect = 0
    
    for index, (testname, getargs, testanswer, expected, fn_name, type) in enumerate(tests):
        dispindex = index+1
        summary = test_summary(dispindex, ntests, fn_name)
        
        try:
            if callable(getargs):
                getargs = getargs()
            
            if type == 'FUNCTION':
                answer = fn_name(*getargs)
            elif type == 'VALUE':
                answer = fn_name
            else:
                answer = [ FN(*getargs) for FN in getargs ]#run_test((index, type, fn_name, getargs), get_lab_module())
        except NotImplementedError:
            print("%d: (%s: Function not yet implemented, NotImplementedError raised)" % (index, testname))
            continue
        except Exception:
            show_exception(summary, testname)
            continue
        
        correct = testanswer(answer, original_val = getargs)
        show_result(summary, testname, correct, answer, expected, verbosity)
        if correct: ncorrect += 1
    
    print("Passed %d of %d tests." % (ncorrect, ntests))
    tests = []
    return ncorrect == ntests

def get_target_upload_filedir():
    """ Get, via user prompting, the directory containing the current lab """
    cwd = os.getcwd() # Get current directory.  Play nice with Unicode pathnames, just in case.
        
    print("Please specify the directory containing your lab.")
    print("Note that all files from this directory will be uploaded!")
    print("Labs should not contain large amounts of data; very-large")
    print("files will fail to upload.")
    print()
    print("The default path is '%s'" % cwd)
    target_dir = raw_input("[%s] >>> " % cwd)

    target_dir = target_dir.strip()
    if target_dir == '':
        target_dir = cwd

    print("Ok, using '%s'." % target_dir)

    return target_dir

def get_tarball_data(target_dir, filename):
    """ Return a binary String containing the binary data for a tarball of the specified directory """
    data = StringIO()
    file = tarfile.open(filename, "w|bz2", data)

    print("Preparing the lab directory for transmission...")
            
    file.add(target_dir+"/lab4.py")
    
    print("Done.")
    print()
    print("The following files have been added:")
    
    for f in file.getmembers():
        print(f.name)
            
    file.close()

    return data.getvalue()
    

def test_online(verbosity=1):
    """ Run online unit tests.  Run them against the 6.034 server via XMLRPC. """
    lab = get_lab_module()

    try:
        server = xmlrpclib.Server(server_url, allow_none=True)
        #print("Getting tests:", (username, password, lab.__name__))
        tests = server.get_tests(username, password, lab.__name__)
        #print("*** TESTS:")
        #print(tests)

    except NotImplementedError: # Solaris Athena doesn't seem to support HTTPS
        print("Your version of Python doesn't seem to support HTTPS, for")
        print("secure test submission.  Would you like to downgrade to HTTP?")
        print("(note that this could theoretically allow a hacker with access")
        print("to your local network to find your 6.034 password)")
        answer = raw_input("(Y/n) >>> ")
        if len(answer) == 0 or answer[0] in "Yy":
            server = xmlrpclib.Server(server_url.replace("https", "http"))
            tests = server.get_tests(username, password, lab.__name__)
        else:
            print("Ok, not running your tests.")
            print("Please try again on another computer.")
            print("Linux Athena computers are known to support HTTPS,")
            print("if you use the version of Python in the 'python' locker.")
            sys.exit(0)
            
    ntests = len(tests)
    ncorrect = 0

    lab = get_lab_module()
    
    target_dir = get_target_upload_filedir()

    tarball_data = get_tarball_data(target_dir, "lab%s.tar.bz2" % lab.LAB_NUMBER)
            
    print("Submitting to the 6.034 Webserver...")

    server.submit_code(username, password, lab.__name__, xmlrpclib.Binary(tarball_data))

    print("Done submitting code.")
    print("Running test cases...")
    
    for index, testcode in enumerate(tests):
        dispindex = index+1
        summary = test_summary(dispindex, ntests, testcode)

        try:
            answer = run_test(testcode, get_lab_module())
        except Exception:
            show_exception(summary, testcode)
            continue

        correct, expected = server.send_answer(username, password, lab.__name__, testcode[0], type_encode(answer))
        show_result(summary, testcode, correct, answer, expected, verbosity)
        if correct: ncorrect += 1
    
    response = server.status(username, password, lab.__name__)
    print(response)



#if __name__ == '__main__':
#    test_offline()
    
def make_test_counter_decorator():
    #tests = []
    def make_test(getargs, testanswer, expected_val, name = None, type = 'FUNCTION'):
        if name != None:
            getargs_name = name
        elif not callable(getargs):
            getargs_name = "_".join(getargs[:-8].split('_')[:-1])
            getargs = lambda: getargs
        else:
            getargs_name = "_".join(getargs.__name__[:-8].split('_')[:-1])
            
        tests.append( ( getargs_name,
                        getargs,
                        testanswer,
                        expected_val,
                        getargs_name,
                        type ) )

    def get_tests():
        return tests

    return make_test, get_tests


make_test, get_tests = make_test_counter_decorator()


# Tests

In [ ]:
# This code implements some very rudimentary matrix-like and vector-like operations
# It is used by the tester.
# You are welcome to use these functions for the laboratory implementation as well,
# though this isn't expected to be necessary.

import random
import operator
import math

def test_code():
    try:
        all([])
    except NameError:
        # The all() function was introduced in Python 2.5.
        # Provide our own implementation if it's not available here.
        def all(iterable):
            for element in iterable:
                if not element:
                    return False
            return True


    def unit_vector(vec1, vec2):
        """ Return a unit vector pointing from vec1 to vec2 """
        diff_vector = map(operator.sub, vec2, vec1)
        
        scale_factor = math.sqrt( sum( map( lambda x: x**2, diff_vector ) ) )
        if scale_factor == 0:
            scale_factor = 1 # We don't have an actual vector, it has zero length
        return map(lambda x: x/scale_factor, diff_vector)

    def vector_compare(vec1, vec2, delta):
        """ Compare two vectors
        Confirm that no two corresponding fields differ by more than delta """
        return all( map(lambda x,y: (abs(x-y) < delta), vec1, vec2) )
        
    def validate_euclidean_distance(list1, list2, dist):
        """
        Confirm that the given distance is the Euclidean distance
        between list1 and list2 by establishing a unit vector between
        the two lists and seeing if vec * dist + list1 == list2
        """

        vec = unit_vector(list1, list2)
        target = map(lambda jmp, base: jmp * dist + base, vec, list1)
        return vector_compare(target, list2, 0.01)

    def random_list(length):
        return [ random.randint(1,100) for x in range(length) ]


    import operator
    import math

    # CSP tests


    EXPECTED_FC_MOOSE_TREE = """ROOT
        1=Mc(c,1)
            2=Y(c,2)
                3=M(c,3)
                    4=O(c,4)
                        5=B(f,5)
                    4=B(c,6)
                        5=O(f,7)
                3=O(c,8)
                    4=M(f,9)
                    4=B(c,10)
                        5=M(f,11)
                3=B(c,12)
                    4=M(f,13)
                    4=O(c,14)
                        5=M(f,15)
            2=M(c,16)
                3=Y(c,17)
                    4=O(c,18)
                        5=B(f,19)
                    4=B(c,20)
                        5=O(f,21)
                3=O(c,22)
                    4=Y(f,23)
                    4=B(c,24)
                        5=Y(c,25)
                            6=P(*,26)
                3=B(u,-)
            2=P(u,-)
    """

    EXPECTED_FCPS_MOOSE_TREE = """ROOT
        1=Mc(c,1)
            2=Y(c,2)
                3=M(f,3)
                3=O(f,4)
                3=B(f,5)
            2=M(c,6)
                3=Y(f,7)
                3=O(c,8)
                    4=B(c,9)
                        5=Y(c,10)
                            6=P(*,11)
                3=B(u,-)
            2=P(u,-)
    """

    def csp_test_1_getargs():
        return [ "moose_csp_problem", "forward_checking" ]

    def csp_test_1_testanswer(val, original_val = None):
        return ( val == EXPECTED_FC_MOOSE_TREE )

    make_test(type = 'FUNCTION',
            getargs = csp_test_1_getargs,
            testanswer = csp_test_1_testanswer,
            expected_val = EXPECTED_FC_MOOSE_TREE,
            name = csp_solver_tree
            )

    def csp_test_2_getargs():
        return [ "moose_csp_problem", "forward_checking_prop_singleton" ]

    def csp_test_2_testanswer(val, original_val = None):
        return ( val == EXPECTED_FCPS_MOOSE_TREE )

    make_test(type = 'FUNCTION',
            getargs = csp_test_2_getargs,
            testanswer = csp_test_2_testanswer,
            expected_val = EXPECTED_FCPS_MOOSE_TREE,
            name = csp_solver_tree
            )

    # KNN tests

    def euclidean_distance_1_getargs():
        return [ [1,2,3], [4,5,6] ]

    def euclidean_distance_1_testanswer(val, original_val = None):
        return ( abs(val - math.sqrt(27)) < 0.00001 )

    make_test(type = 'FUNCTION',
            getargs = euclidean_distance_1_getargs,
            testanswer = euclidean_distance_1_testanswer,
            expected_val = math.sqrt(27),
            name = euclidean_distance
            )


    senate_people = read_congress_data('S110.ord')
    senate_votes = read_vote_data('S110desc.csv')

    house_people = read_congress_data('H110.ord')
    house_votes = read_vote_data('H110desc.csv')

    last_senate_people = read_congress_data('S109.ord')
    last_senate_votes = read_vote_data('S109desc.csv')


    def euclidean_distance_1_getargs():
        return [ [1,2,3], [4,5,6] ]

    def euclidean_distance_1_testanswer(val, original_val = None):
        return ( abs(val - math.sqrt(27)) < 0.00001 )

    make_test(type = 'FUNCTION',
            getargs = euclidean_distance_1_getargs,
            testanswer = euclidean_distance_1_testanswer,
            expected_val = math.sqrt(27),
            name = euclidean_distance
            )

    def euclidean_distance_2_getargs():
        return [ [0,0], [3,4] ]

    def euclidean_distance_2_testanswer(val, original_val = None):
        return ( abs(val - math.sqrt(25)) < 0.00001 )

    make_test(type = 'FUNCTION',
            getargs = euclidean_distance_2_getargs,
            testanswer = euclidean_distance_2_testanswer,
            expected_val = 5,
            name = euclidean_distance
            )

    def disorder_1_getargs():
        return [ ['Democrat','Democrat','Democrat'], ['Republican',"Republican"]]
    def disorder_1_testanswer(val, original_val=None):
        return (abs(val - 0) < 0.0001)
    make_test(type="FUNCTION",
            getargs = disorder_1_getargs,
            testanswer = disorder_1_testanswer,
            expected_val = 0,
            name=information_disorder)

    def disorder_2_getargs():
        return [ ['Democrat','Republican'], ['Democrat',"Republican"]]
    def disorder_2_testanswer(val, original_val=None):
        return (abs(val - 1) < 0.0001)
    make_test(type="FUNCTION",
            getargs = disorder_2_getargs,
            testanswer = disorder_2_testanswer,
            expected_val = 1,
            name=information_disorder)

    def disorder_3_getargs():
        return [ ['Democrat','Democrat','Democrat','Republican'],
                ['Democrat','Republican',"Republican"]]
    def disorder_3_testanswer(val, original_val=None):
        return (abs(val - 0.8571428) < 0.0001)
    make_test(type="FUNCTION",
            getargs = disorder_3_getargs,
            testanswer = disorder_3_testanswer,
            expected_val = 0.8571428,
            name=information_disorder)


    def eval_test_1_getargs():
        senate_group1, senate_group2 = crosscheck_groups(senate_people)
        return [ 'my_classifier', senate_group1, senate_group2 ]

    def eval_test_1_testanswer(val, original_val = None):
        return ( val >= 97 )

    make_test(type = 'FUNCTION',
            getargs = eval_test_1_getargs,
            testanswer = eval_test_1_testanswer,
            expected_val = "Less than or equal to 3 miscategorizations",
            name = eval_test
            )

    rep_classified_getargs = "rep_classified"

    def rep_classified_testanswer(val, original_val = None):
        return ( val >= 430 )

    make_test(type = 'VALUE',
            getargs = rep_classified_getargs,
            testanswer = rep_classified_testanswer,
            expected_val = "430 or larger",
            name = rep_classified
            )

    senator_classified_getargs = "senator_classified"

    def senator_classified_testanswer(val, original_val = None):
        return ( val >= 90 )

    make_test(type = 'VALUE',
            getargs = senator_classified_getargs,
            testanswer = senator_classified_testanswer,
            expected_val = "90 or larger",
            name = senator_classified
            )

    old_senator_classified_getargs = "old_senator_classified"

    def old_senator_classified_testanswer(val, original_val = None):
        return ( val >= 95 )

    make_test(type = 'VALUE',
            getargs = old_senator_classified_getargs,
            testanswer = old_senator_classified_testanswer,
            expected_val = "95 or larger",
            name = old_senator_classified
            )

    test_offline()

#**Test your code**

In [ ]:
test_code()

Test 1/12 (<function csp_solver_tree at 0x7f4f3de50f28>): Incorrect.
	 <function csp_solver_tree at 0x7f4f3de50f28>
Got:      ROOT
    1=Mc(c,1)
        2=Y(c,2)
            3=M(c,3)
                4=O(c,4)
                    5=B(f,5)
                4=B(c,6)
                    5=O(f,7)
            3=O(c,8)
                4=M(f,9)
                4=B(c,10)
                    5=M(f,11)
            3=B(c,12)
                4=M(f,13)
                4=O(c,14)
                    5=M(f,15)
        2=M(c,16)
            3=Y(c,17)
                4=O(c,18)
                    5=B(f,19)
                4=B(c,20)
                    5=O(f,21)
            3=O(c,22)
                4=Y(f,23)
                4=B(c,24)
                    5=Y(c,25)
                        6=P(*,26)
            3=B(u,-)
        2=P(u,-)

Expected: ROOT
        1=Mc(c,1)
            2=Y(c,2)
                3=M(c,3)
                    4=O(c,4)
                        5=B(f,5)
                    4=B(c,6)
    